# Convert EBM to IRM

This notebook takes the two-layer energy balance model tunings from Donald Cummins and converts them to a two-layer impulse response function.

It will then save these into a CSV file.

In [ ]:
import os

import numpy as np
import pandas as pd
import scipy.linalg
from tqdm import tqdm

from fair.constants import DOUBLING_TIME_1PCT
from fair.earth_params import seconds_per_year, earth_radius
from fair.forcing.ghg import meinshausen2020
from fair.energy_balance_model import EnergyBalanceModel

In [ ]:
df = pd.read_csv(
    os.path.join("..", "data", "calibration", "4xCO2_cummins_ebm2.csv")
)

In [ ]:
models = df['model'].unique()
models

In [ ]:
params = {}
for model in models:
    params[model] = {}
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        params[model][run] = {}
        params[model][run]['gamma_autocorrelation'] = df.loc[condition, 'gamma'].values[0]
        params[model][run]['ocean_heat_capacity'] = df.loc[condition, 'C1':'C2'].values.squeeze()
        params[model][run]['ocean_heat_transfer'] = df.loc[condition, 'kappa1':'kappa2'].values.squeeze()
        params[model][run]['deep_ocean_efficacy'] = df.loc[condition, 'epsilon'].values[0]
        params[model][run]['sigma_eta'] = df.loc[condition, 'sigma_eta'].values[0]
        params[model][run]['sigma_xi'] = df.loc[condition, 'sigma_xi'].values[0]
        params[model][run]['forcing_4co2'] = df.loc[condition, 'F_4xCO2'].values[0]

In [ ]:
params

In [ ]:
co2 = 284.3169988
ch4 = 808.2490285
n2o = 273.021047

double_co2 = co2 * 2
quadruple_co2 = co2 * 4

rf_4co2 = meinshausen2020(
    np.array([4*co2, ch4, n2o]).reshape((1, 1, 1, 3)),
    np.array([co2, ch4, n2o]).reshape((1, 1, 1, 3)),
    np.array([1, 1, 1]).reshape((1, 1, 1, 3)), 
    np.ones((1, 1, 1, 3)),
    0, 1, 2, []
).squeeze()[0]

rf_2co2 = meinshausen2020(
    np.array([2*co2, ch4, n2o]).reshape((1, 1, 1, 3)),
    np.array([co2, ch4, n2o]).reshape((1, 1, 1, 3)),
    np.array([1, 1, 1]).reshape((1, 1, 1, 3)), 
    np.ones((1, 1, 1, 3)),
    0, 1, 2, []
).squeeze()[0]

forcing_2co2_4co2_ratio=rf_2co2/rf_4co2
forcing_2co2_4co2_ratio

In [ ]:
rf_2co2*1.05

In [ ]:
ebm = EnergyBalanceModel(**params['ACCESS-CM2']['r1i1p1f1'])

In [ ]:
params['ACCESS-CM2']['r1i1p1f1']

In [ ]:
ebm._eb_matrix()

In [ ]:
ebm.emergent_parameters(forcing_2co2_4co2_ratio = forcing_2co2_4co2_ratio)

In [ ]:
ebm.ecs

In [ ]:
for model in models:
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        ebm = EnergyBalanceModel(**params[model][run])
        ebm.emergent_parameters()
        params[model][run] = ebm.__dict__

In [ ]:
params

In [ ]:
# reconstruct a data table and save
df_out = pd.DataFrame(columns=['model', 'run', 'ecs', 'tcr', 'tau1', 'tau2', 'q1', 'q2'])


count = 0
for model in models:
    for run in df.loc[df['model']==model, 'run']:
        values_to_add = {
            'model': model,
            'run': run,
            'ecs': params[model][run]['ecs'],
            'tcr': params[model][run]['tcr'],
            'tau1': params[model][run]['timescales'][0],
            'tau2': params[model][run]['timescales'][1],
            'q1': params[model][run]['response_coefficients'][0],
            'q2': params[model][run]['response_coefficients'][1],
        }
        row_to_add = pd.Series(values_to_add, name=count)
        df_out = df_out.append(row_to_add)
        count = count + 1

In [ ]:
df_out.sort_values(['model', 'run'], inplace=True)

In [ ]:
df_out.to_csv(os.path.join("..", "data", "calibration", "4xCO2_impulse_response_ebm2.csv"), index=False)

## While we're here, sort and de-index the energy balance data from the R script

Why? Makes comparing the two CSVs easier

Save in a new file, as earlier on we use the R-generated one to read in

In [ ]:
df.sort_values(['model', 'run'], inplace=True)

In [ ]:
df.to_csv(os.path.join("..", "data", "calibration", "4xCO2_energy_balance_ebm2.csv"), index=False)